# Notebook Parameter Estimation

The pourpose of this notebook is to estimate the parameters of a of three-phase induction motor.

In [290]:
import math
import cmath
import yaml

## Book Keeping

In [291]:
MODEL = "3GBA132210"
VERBOSITY = 1

## Relevant Test Report Parameters

In [292]:
LINE_RESISTANCE = [1.21878, 1.21939, 1.21817]
FREQUENCY = 50
MOMENT_OF_INERTIA = 0.0297
LINE_VOLTAGE_NO_LOAD = 400
INPUT_CURRENT_NO_LOAD = 4.70
POWER_FACTOR_NO_LOAD = 0.06
LINE_VOLTAGE_BLOCKED_ROTOR = 66
INPUT_CURRENT_BLOCKED_ROTOR = 10.50
POWER_FACTOR_BLOCKED_ROTOR = 0.36
ROTOR_SPEED_75_PERCENT_LOAD = 1469
SYNCHRONOUS_SPEED = 1500
OUTPUT_POWER_75_PERCENT_LOAD = 4.12e3
LINE_VOLTAGE_75_PERCENT_LOAD = 400
INPUT_CURRENT_75_PERCENT_LOAD = 8.65
POWER_FACTOR_75_PERCENT_LOAD = 0.76
LINE_VOLTAGE_100_PERCENT_LOAD = 400
INPUT_CURRENT_100_PERCENT_LOAD = 10.90
POWER_FACTOR_100_PERCENT_LOAD = 0.81
NUMBER_OF_POLES = 4

In [293]:
"""
LINE_RESISTANCE = [0.3970, 0.3930, 0.3947]
FREQUENCY = 50
MOMENT_OF_INERTIA = 0.095
LINE_VOLTAGE_NO_LOAD = 401
INPUT_CURRENT_NO_LOAD = 11.7
POWER_FACTOR_NO_LOAD = 0.06
LINE_VOLTAGE_BLOCKED_ROTOR = 74
INPUT_CURRENT_BLOCKED_ROTOR = 28.7
POWER_FACTOR_BLOCKED_ROTOR = 0.35
ROTOR_SPEED_75_PERCENT_LOAD = 1476
SYNCHRONOUS_SPEED = 1500
OUTPUT_POWER_75_PERCENT_LOAD = 11.35e3
LINE_VOLTAGE_75_PERCENT_LOAD = 400
INPUT_CURRENT_75_PERCENT_LOAD = 22.8
POWER_FACTOR_75_PERCENT_LOAD = 0.79
LINE_VOLTAGE_100_PERCENT_LOAD = 400
INPUT_CURRENT_100_PERCENT_LOAD = 28.7
POWER_FACTOR_100_PERCENT_LOAD = 0.83
NUMBER_OF_POLES = 4
"""

'\nLINE_RESISTANCE = [0.3970, 0.3930, 0.3947]\nFREQUENCY = 50\nMOMENT_OF_INERTIA = 0.095\nLINE_VOLTAGE_NO_LOAD = 401\nINPUT_CURRENT_NO_LOAD = 11.7\nPOWER_FACTOR_NO_LOAD = 0.06\nLINE_VOLTAGE_BLOCKED_ROTOR = 74\nINPUT_CURRENT_BLOCKED_ROTOR = 28.7\nPOWER_FACTOR_BLOCKED_ROTOR = 0.35\nROTOR_SPEED_75_PERCENT_LOAD = 1476\nSYNCHRONOUS_SPEED = 1500\nOUTPUT_POWER_75_PERCENT_LOAD = 11.35e3\nLINE_VOLTAGE_75_PERCENT_LOAD = 400\nINPUT_CURRENT_75_PERCENT_LOAD = 22.8\nPOWER_FACTOR_75_PERCENT_LOAD = 0.79\nLINE_VOLTAGE_100_PERCENT_LOAD = 400\nINPUT_CURRENT_100_PERCENT_LOAD = 28.7\nPOWER_FACTOR_100_PERCENT_LOAD = 0.83\nNUMBER_OF_POLES = 4\n'

In [294]:
Rs = (sum(LINE_RESISTANCE) / len(LINE_RESISTANCE)) / 2

if VERBOSITY:
    print("Rs :: {:.4e}".format(Rs))

Rs :: 6.0939e-01


In [295]:
Zbr = (LINE_VOLTAGE_BLOCKED_ROTOR / math.sqrt(3)) / INPUT_CURRENT_BLOCKED_ROTOR

Xs = 0.5 * Zbr * math.sin(math.acos(POWER_FACTOR_BLOCKED_ROTOR))
Ls = Xs / (SYNCHRONOUS_SPEED * (math.pi / 30) * (NUMBER_OF_POLES / 2))

if VERBOSITY:
    print("Ls :: {:.4e}".format(Ls))

Ls :: 5.3886e-03


In [296]:
Vsnl = LINE_VOLTAGE_NO_LOAD / math.sqrt(3)
Isnl = cmath.rect(INPUT_CURRENT_NO_LOAD, -math.acos(POWER_FACTOR_NO_LOAD))
Snl = Vsnl * Isnl.conjugate()
Vmsl = Vsnl - Isnl * complex(Rs, Xs)

Xm = abs(Vmsl)**2 / (Snl.imag - abs(Isnl)**2 * Xs)
Rm = abs(Vmsl)**2 / (Snl.real - abs(Isnl)**2 * Rs)

Lm = Xm / (SYNCHRONOUS_SPEED * (math.pi / 30) * (NUMBER_OF_POLES / 2))

if VERBOSITY:
    print("Rm :: {:.4e}".format(Rm))
    print("Lm :: {:.4e}".format(Lm))

Rm :: 9.6116e+02
Lm :: 1.5110e-01


In [297]:
s75 = (SYNCHRONOUS_SPEED - ROTOR_SPEED_75_PERCENT_LOAD) / SYNCHRONOUS_SPEED

Vs75 = LINE_VOLTAGE_75_PERCENT_LOAD / math.sqrt(3)
Is75 = cmath.rect(INPUT_CURRENT_75_PERCENT_LOAD, -math.acos(POWER_FACTOR_75_PERCENT_LOAD))
S75 = Vs75 * Is75.conjugate()
Vm75 = Vs75 - Is75 * complex(Rs, Xs)
Im75 = Vm75 / (complex(0, Rm * Xm) / complex(Rm, Xm))
Ir75 = Is75 - Im75

Rr = ((S75.real - abs(Vm75)**2 / Rm - abs(Is75)**2 * Rs) * s75) / abs(Ir75)**2
Xr = (S75.imag - abs(Vm75)**2 / Xm - abs(Is75)**2 * Xs) / abs(Ir75)**2

Lr = Xr / (SYNCHRONOUS_SPEED * (math.pi / 30) * (NUMBER_OF_POLES / 2))

if VERBOSITY:
    print("Rr :: {:.4e}".format(Rr))
    print("Lr :: {:.4e}".format(Lr))

Rr :: 6.7702e-01
Lr :: 1.2792e-02


In [298]:
Pe75 = 3 * (abs(Ir75)**2 * Rr * ((1 - s75) / s75))
Br = (Pe75 - OUTPUT_POWER_75_PERCENT_LOAD) / (ROTOR_SPEED_75_PERCENT_LOAD * (math.pi / 30))**2

if VERBOSITY:
    print("Br :: {:.4e}".format(Br))

Br :: 2.6131e-03


In [299]:
Vs100 = LINE_VOLTAGE_100_PERCENT_LOAD / math.sqrt(3)
Is100 = cmath.rect(INPUT_CURRENT_100_PERCENT_LOAD, -math.acos(POWER_FACTOR_100_PERCENT_LOAD))
S100 = Vs100 * Is100.conjugate()
Vm100 = Vs100 - Is100 * complex(Rs, Xs)
Im100 = Vm100 / (complex(0, Rm * Xm) / complex(Rm, Xm))
Ir100 = Is100 - Im100
Pe100 = 3 * ((Vm100 - Ir100 * complex(Rs, Xs)) * Ir100.conjugate()).real
s100 = (3 * abs(Ir100)**2 * Rr) / ((3 * abs(Ir100)**2 *Rr) + Pe100)
wr100 = (1 - s100) * SYNCHRONOUS_SPEED
Pmec100 = Pe100 - (Br * (wr100 * (math.pi / 30))**2)
n = (Pmec100 / (3 * S100.real))

if VERBOSITY:
    print("Ir[A] :: {:.4e}".format(abs(Ir100)))
    print("n[r/min] :: {:.4e}".format(wr100))
    print("P2[kW] :: {:.4e}".format(Pmec100))
    print("n[%] :: {:.4e}".format(n))

Ir[A] :: 9.0313e+00
n[r/min] :: 1.4569e+03
P2[kW] :: 5.5455e+03
n[%] :: 9.0658e-01


In [300]:
with open("../reports/" + MODEL + "_parameters.yaml", "w") as file:
    documents = yaml.dump({
            "Vs": LINE_VOLTAGE_NO_LOAD / math.sqrt(3),
            "Jr": MOMENT_OF_INERTIA,
            "we": (2 * math.pi) * FREQUENCY,
            "P": NUMBER_OF_POLES,
            "Rs": Rs,
            "Ls": Ls,
            "Rm": Rm,
            "Lm": Lm,
            "Lr": Lr,
            "Rr": Rr,
            "Br": Br,
            "Ir": abs(Ir100),
        }, 
        file
    )